In [5]:
import transformers
from transformers import BertConfig, BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup,DistilBertModel, DistilBertConfig
import torch
import numpy as np
import pandas as pd
from torch import nn

In [6]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
class_names = ['negative', 'positive']
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [8]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)
model = torch.load('model.bin')

/Users/shafisourov/anaconda3/envs/shafi_env/lib/python3.6/site-packages/torch/serialization.py:559: UserWarning: Couldn't retrieve source code for container of type SentimentClassifier. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


In [9]:
df = pd.read_csv("meme_img_des_caption_210310.csv")
def to_sentiment(sentiment):
  if sentiment == 'Non-offensiv':
    return 1
  else:
    return 0

def con_img_des_img_cap(img_des):
    return img_des

df['Sentiment'] = df.Sentiment.apply(to_sentiment)
df.head()
df['text'] = df.Image_text.apply(con_img_des_img_cap) + ' '+ df.image_caption.apply(con_img_des_img_cap)

In [10]:
# tokens = tokenizer.tokenize(review_text)
# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# tokenizer.convert_ids_to_tokens(token_ids)

In [11]:
for i in range(12,20):
    review_text , sentiment= df.text[i], df.Sentiment[i]
    
    MAX_LEN = 160
    encoded_review = tokenizer.encode_plus(
      review_text,
      max_length=MAX_LEN,
      add_special_tokens=True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    print(review_text)
    print('Ground truth',class_names[int(sentiment)])
    print('Predicted Sentiment  : ',class_names[prediction])

V FULL SPEECH : Donald Trump Republican National Convent Et ABC15 Arizona 67,247 view aoC REPUBLICAN NATIONAL CONVENTION ARIZONA 57:42 1:16:41 a group of people
Ground truth positive
Predicted Sentiment  :  positive


Google News & Weather 10:32 PM Bernie Sanders , Vowing to Stay in Race , Ass . GE New York Times a smartphone notification
Ground truth positive
Predicted Sentiment  :  positive


L Democratic Party Republican Party X Cocktail Party  a ballot paper
Ground truth positive
Predicted Sentiment  :  positive


MAGAZINE SEARCH EMAIL SIGNUP poumoo.coM 8 f OUR LATEST I 'M WITH ECER Add my name 2016 Think You 've Got It Locked , Hillary ? Meet Jill Stein . The Green Party candidate insists it 's her year to get noticed and she may make it onto 47 state ballots . By BILL SCHER I June 19 , 2016 Getty Images a women sideface
Ground truth positive
Predicted Sentiment  :  positive


BRACE YOURSELF DISAPPOINTED BERNIE SANDERS SUPPORTERS ARE COMING made on inngur  a picture of a man holding a bear
Ground truth negative
Predicted Sentiment  :  negative


19 hrs None of your opinions matter Like Comment Share and 7 others Yeah , but that 's like , your opinion Like Reply 12 hrs Also it 's absolutely a fact that your like 9 paragraph soliloquy about black lives matter or Bernie sanders or something that you put on Facebook does n't matter Like Reply 3 hrs Especially because everyone seems to think their Facebook E profile is the fucking mouth of god Like Reply 3 hrs a picture of a picture of a book
Ground truth positive
Predicted Sentiment  :  positive


Lady GARDEN Liberal Democrat  a women
Ground truth negative
Predicted Sentiment  :  negative
Bernie or Hillary ? Be informed . Compare them on the issues that matter . Issue : Venom An incredible band that 's one A toxin injected by of the most influence metal an animal from a bands of all time . With out them there would be no sting or bite . black , thrash and death metal . bernieorhillary.jaxbot me  a poster with 2 people
Ground truth positive
Predicted Sentiment  :  positive


In [12]:
# history = model.history
# plt.plot(history['train_acc'], label='train accuracy')
# plt.plot(history['val_acc'], label='validation accuracy')
# plt.title('Training history')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend()
# plt.ylim([0, 1]);

AttributeError: 'SentimentClassifier' object has no attribute 'history'